<a href="https://colab.research.google.com/github/i-saw/Giga-chat-voice/blob/main/%D0%9F%D0%BE%D0%BC%D0%BE%D1%89%D0%BD%D0%B8%D1%86%D0%B0_%D0%93%D0%B8%D0%B3%D0%B0_%D1%81_%D0%B3%D0%BE%D0%BB%D0%BE%D1%81%D0%BE%D0%B2%D1%8B%D0%BC_%D0%BE%D0%B1%D1%89%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Гигачат с голосом
[Репозиторий библиотеки](https://github.com/ai-forever/gigachain)

[Документация на сайте Сбера](https://developers.sber.ru/docs/ru/gigachat/sdk/overview)


In [ ]:
!pip install gigachain
!pip install pydub
!pip install SpeechRecognition
!pip install gTTS
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 995.3/995.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━

***Импортируем ключ, который получаем на сайте***: https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart

In [ ]:
# импортируем ключ, который получаем на сайте . SBER_AUTH и ключ вводим слева в меню секреты (значок выглядит как ключ)
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

In [ ]:
# код для распознавания речи
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from pydub import AudioSegment
import speech_recognition as sr
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_and_convert(sec=10, max_attempts=3):
    attempts = 0

    while attempts < max_attempts:
        display(Javascript(RECORD))
        s = output.eval_js('record(%d)' % (sec*1000))
        b = b64decode(s.split(',')[1])
        with open('audio.webm', 'wb') as f:
            f.write(b)

        sound = AudioSegment.from_file('audio.webm', format="webm")
        sound.export('audio.wav', format="wav")

        # Создаем объект Recognizer
        r = sr.Recognizer()

        # Открываем аудиофайл с помощью объекта AudioFile
        audio_file = "/content/audio.wav"
        with sr.AudioFile(audio_file) as source:
            audio_data = r.record(source)  # Записываем аудиоданные в объект AudioData

            # Используем метод recognize_google для распознавания речи на русском языке
            try:
                text = r.recognize_google(audio_data, language="ru-RU")
                return str(text)  # Возвращаем результат распознавания в случае успеха
            except sr.UnknownValueError:
                attempts += 1  # Увеличиваем количество попыток при неудачном распознавании
                continue
            except sr.RequestError as e:
                text = "Ошибка сервиса Google Speech Recognition"
                return text  # Возвращаем текст ошибки в случае проблем с сервисом

    return 'Стоп'  # Останавливаем беседу


In [ ]:
# код для синтеза речи
from gtts import gTTS
from IPython.display import Audio, display
from pydub import AudioSegment
import os

def text_to_speech(text, lang='ru', speed=1.25):
    # Создаем объект gTTS с текстом, языком и скоростью воспроизведения
    tts = gTTS(text=text, lang=lang, slow=False)

    # Сохраняем аудиофайл в формате mp3
    tts.save("/content/output.mp3")

    # Загрузка аудиофайла
    audio = AudioSegment.from_file("/content/output.mp3")

    # Увеличение скорости воспроизведения
    audio_with_altered_speed = audio.speedup(playback_speed=speed)

    # Сохранение измененного аудиофайла
    audio_with_altered_speed.export("/content/output_speedup.mp3", format="mp3")

    # Воспроизведение измененного аудиофайла
    display(Audio("/content/output_speedup.mp3", autoplay=True))

    # Удаляем временный аудиофайл
    os.remove("/content/output_speedup.mp3")

# Пример использования
# text = "Привет, как дела?"
# text_to_speech(text, lang='ru', speed=1.25)


In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

In [ ]:
giga = GigaChat(credentials=auth,
                model='GigaChat:latest',
                verify_ssl_certs=False
                )

In [ ]:
# создадим список, где будем хранить сообщения
# системное сообщение идет первым в списке (если, конечно, оно есть)

msgs = [
    SystemMessage(content='Отвечай как помощница Гига.')
]

Запустим помощницу и поговорим с ней. Скажите "Стоп" когда захотите остановиться

In [ ]:
while True:
  user_input = ("Пользователь: " + record_and_convert(sec=10, max_attempts=20))
  if user_input == 'Пользователь: стоп':
    print("Ну вот и пообщались)")
    break
  print(user_input)
  msgs.append(HumanMessage(content=user_input))
  answer = giga(msgs)
  msgs.append(answer)
  print('Гига:', answer.content)
  text_to_speech(answer.content, lang='ru', speed=1.25)


<IPython.core.display.Javascript object>

Пользователь: Кто ты
Гига: Я — виртуальный помощник, созданный для того, чтобы помогать людям в решении различных задач.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Пользователь: Назови семь чудес света
Гига: Семь чудес света древнего мира включают: Великую пирамиду Хеопса, Висячие сады Семирамиды, Статую Зевса в Олимпии, Мавзолей в Галикарнасе, Колосс Родосский, Храм Артемиды в Эфесе и Александрийский маяк.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ну вот и пообщались)
